In [9]:
from google import genai
import os
import requests 
from google.genai import types

In [11]:
from google.genai import types

# Define the function declaration properly
weather_function = types.FunctionDeclaration(
    name="get_weather",
    description="Fetch real-time weather information for a given city. Use this tool whenever the user asks about weather, temperature, humidity, or climate of a place.",
    parameters={
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Name of the city to fetch weather information for."
            }
        },
        "required": ["city"]
    }
)


In [6]:
def get_weather(city: str):
    API_KEY = os.getenv("OPEN_WEATHER_MAP")
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    return data

In [7]:
client = genai.Client()

In [13]:
tools = types.Tool(function_declarations=[weather_function])

In [14]:
config = types.GenerateContentConfig(tools=[tools])


In [15]:
contents = [
    types.Content(
        role="user", parts=[types.Part(text="What is the weather of Kathmandu")]
    )
]

In [17]:
response = client.models.generate_content(
    model= "gemini-2.5-flash",
    contents = contents,
    config = config
)

In [19]:
print(response.candidates[0].content.parts[0].function_call)


id=None args={'city': 'Kathmandu'} name='get_weather'


In [20]:
tool_call = response.candidates[0].content.parts[0].function_call


In [21]:
if tool_call.name == "get_weather":
    result = get_weather(**tool_call.args)
    print(f"Function execution result: {result}")

Function execution result: {'coord': {'lon': 85.3167, 'lat': 27.7167}, 'weather': [{'id': 721, 'main': 'Haze', 'description': 'haze', 'icon': '50d'}], 'base': 'stations', 'main': {'temp': 17.12, 'feels_like': 16.32, 'temp_min': 17.12, 'temp_max': 17.12, 'pressure': 1016, 'humidity': 55, 'sea_level': 1016, 'grnd_level': 854}, 'visibility': 5000, 'wind': {'speed': 1.54, 'deg': 50}, 'clouds': {'all': 75}, 'dt': 1765089710, 'sys': {'type': 1, 'id': 9201, 'country': 'NP', 'sunrise': 1765069000, 'sunset': 1765106635}, 'timezone': 20700, 'id': 1283240, 'name': 'Kathmandu', 'cod': 200}


In [22]:
function_response_part = types.Part.from_function_response(
    name=tool_call.name,
    response={"result": result},
)

In [23]:
function_response_part

Part(
  function_response=FunctionResponse(
    name='get_weather',
    response={
      'result': <dict len=13>
    }
  )
)

In [24]:
contents.append(response.candidates[0].content) # Append the content from the model's response.
contents.append(types.Content(role="user", parts=[function_response_part]))

In [25]:
final_response = client.models.generate_content(
    model="gemini-2.5-flash",
    config=config,
    contents=contents,
)

In [ ]:
print(final_response.text)

The weather in Kathmandu is currently hazy with a temperature of 17.12°C. The humidity is 55% and the wind speed is 1.54 m/s at 50 degrees.
